Miami 311 service call dataset taken from here https://datahub-miamigis.opendata.arcgis.com/datasets/7cc10915ede14bb58be312413842a4ce_0/explore

In [1]:
import pandas as pd

df = pd.read_csv('./City_of_Miami_311_Service_Requests_Since_2015.csv')
print(df.shape)
print(df.columns)
print(df['ticket_created_date_time'].head(1))
print(df['ticket_created_date_time'].tail(1))

(108547, 29)
Index(['X', 'Y', 'ticket_id', 'issue_type', 'issue_Description', 'case_owner',
       'street_address', 'city', 'state', 'zip_code', 'District',
       'created_year_month', 'ticket_created_date_time',
       'ticket_last_updated_date_time', 'ticket_closed_date_time',
       'ticket_status', 'x_coordinate', 'y_coordinate', 'latitude',
       'longitude', 'coordinates', 'method_received', 'goal_days',
       'actual_completed_days', 'Property_Folio_Number', 'Unit_Number',
       'Ticket_Priority', 'OverDueFlag', 'ObjectId'],
      dtype='object')
0    2020/10/01 06:48:47.606+00
Name: ticket_created_date_time, dtype: object
108546    2023/04/11 17:52:42.039+00
Name: ticket_created_date_time, dtype: object


Number of unique issues:

In [2]:
pd.options.display.max_rows = 100
df['issue_Description'].value_counts()

issue_Description
GARBAGE CONTAINER DAMAGED                                     13830
ILLEGAL DUMPING / LITTER                                       8566
ZONING CALL BACK REQUEST                                       6183
RECYCLING MISSED                                               5961
GARBAGE - MISSED                                               5763
CODE ENFORCEMENT CU INSPECTION REQUEST                         5531
FIRE CU INSPECTION REQUEST                                     5349
POTHOLE                                                        5277
SOLID WASTE MISCELLANEOUS REQUESTS                             5052
STORM FLOOD/ DRAINAGE                                          4490
GARBAGE CONTAINER STOLEN                                       4277
TREE TRIM AND ROOT PRUNING                                     4016
RECYCLING CART DAMAGED                                         3657
DEAD ANIMAL PICKUP                                             3036
TRASH HOLE                    

What part of the dataset is garbage/waste related

In [3]:
garbage_issues = [
    'GARBAGE CONTAINER DAMAGED',
    'ILLEGAL DUMPING / LITTER',
    'RECYCLING MISSED',
    'GARBAGE - MISSED',
    'SOLID WASTE MISCELLANEOUS REQUESTS',
    'GARBAGE CONTAINER STOLEN',
    'RECYCLING CART DAMAGED',
    'DEAD ANIMAL PICKUP',
    'TRASH HOLE',
    'BULKY TRASH SPECIAL PICKUP REQUEST TRACKER',
    'BULKY TRASH - MISSED',
    'RECYCLING CART STOLEN',
    'SOLID WASTE RESIDENTIAL NEW SERVICE / ACCOUNT UPDATE',
    'GARBAGE/TRASH/RECYCLING OUT TO EARLY',
]

df_garbage = df.loc[df['issue_Description'].isin(garbage_issues)]
print(sum(df_garbage['issue_Description'].value_counts()), '/', df['issue_Description'].count())

df_non_garbage = df.loc[~df['issue_Description'].isin(garbage_issues)]

df_non_garbage['issue_Description'].value_counts()



59994 / 108547


issue_Description
ZONING CALL BACK REQUEST                                      6183
CODE ENFORCEMENT CU INSPECTION REQUEST                        5531
FIRE CU INSPECTION REQUEST                                    5349
POTHOLE                                                       5277
STORM FLOOD/ DRAINAGE                                         4490
TREE TRIM AND ROOT PRUNING                                    4016
OVERGROWN LOT                                                 2068
SIDEWALK DAMAGE                                               2020
POULTRY (CHICKEN) OR GRAZING ANIMAL                           1011
MIAMI TROLLEY - ON DEMAND TRANSPORTATION SERVICE COMPLAINT     838
COM OTHER                                                      723
TREE BLOCKING TRAFFIC SIGNAL/LIGHT (STOP SIGN)                 658
PARKING - COMMERCIAL VEHICLE ILLEGALLY IN RESIDENTIAL ZONE     627
NOISE VIOLATION                                                616
ILLEGAL STORAGE / JUNK IN YARD              

In [4]:
admin_issues = [
    'ZONING CALL BACK REQUEST',
    'CODE ENFORCEMENT CU INSPECTION REQUEST',
    'FIRE CU INSPECTION REQUEST',
    'ILLEGAL CONSTRUCTION',
    'NO BUSINESS TAX RECEIPT / CERTIFICATE OF USE',
    'ILLEGAL CONSTRUCTION IN PROGRESS -BUILDING',
    'BUILDING DEMOLITIONS AND UNSAFE STRUCTURES',
    'BUILDING MAINTENANCE INADEQUATE',
    'ILLEGAL UNITS',
]

df_admin = df_non_garbage.loc[df['issue_Description'].isin(admin_issues)]

df_non_admin = df_non_garbage.loc[~df['issue_Description'].isin(admin_issues)]

print(sum(df_admin['issue_Description'].value_counts()), '/', df['issue_Description'].count())

print(df_non_admin['issue_Description'].describe())

df_non_admin['issue_Description'].value_counts()

19852 / 108547
count       28701
unique         47
top       POTHOLE
freq         5277
Name: issue_Description, dtype: object


issue_Description
POTHOLE                                                       5277
STORM FLOOD/ DRAINAGE                                         4490
TREE TRIM AND ROOT PRUNING                                    4016
OVERGROWN LOT                                                 2068
SIDEWALK DAMAGE                                               2020
POULTRY (CHICKEN) OR GRAZING ANIMAL                           1011
MIAMI TROLLEY - ON DEMAND TRANSPORTATION SERVICE COMPLAINT     838
COM OTHER                                                      723
TREE BLOCKING TRAFFIC SIGNAL/LIGHT (STOP SIGN)                 658
PARKING - COMMERCIAL VEHICLE ILLEGALLY IN RESIDENTIAL ZONE     627
ILLEGAL STORAGE / JUNK IN YARD                                 616
NOISE VIOLATION                                                616
CITY SHOPPING CART REMOVAL                                     613
GRAFFITI                                                       572
ABANDONED VEHICLE - PRIVATE                 

Code owners:

In [5]:
df['case_owner'].value_counts()

case_owner
Solid Waste                  57748
Resilience & Public Works    22498
Code Compliance              14392
Zoning                        6183
Fire-Rescue                   5349
NET                           1362
Building                       941
Police                          70
Innovation & Technology          4
Name: count, dtype: int64

Resilience & Public Works issues

In [6]:
df_rnpw = df.loc[df['case_owner'] == 'Resilience & Public Works']
df_rnpw['issue_Description'].value_counts()

issue_Description
POTHOLE                                                       5277
STORM FLOOD/ DRAINAGE                                         4490
TREE TRIM AND ROOT PRUNING                                    4016
TRASH HOLE                                                    2504
SIDEWALK DAMAGE                                               2020
MIAMI TROLLEY - ON DEMAND TRANSPORTATION SERVICE COMPLAINT     838
TREE BLOCKING TRAFFIC SIGNAL/LIGHT (STOP SIGN)                 658
SWALE GRADING / DROP OFF REPAIR                                425
TREE DOWN  BLOCKING STREET                                     409
MANHOLE / UTILITY COVER                                        319
TREE DOWN NOT BLOCKING THE STREET                              259
CAVE IN OR SINK HOLE                                           257
TREE STUMP REMOVAL/ GRINDING                                   235
ROAD DEBRIS BLOCKING TRAFFIC RIGHT OF WAY                      202
LANDSCAPING CITY RIGHT OF WAY / MOWING      

In [7]:
pd.options.display.max_columns = 30
df['ticket_created_date_time'] = df['ticket_created_date_time'].apply(lambda val: val.split(' ')[0].replace('/', '-'))
df.loc[df['issue_Description'] == 'STORM FLOOD/ DRAINAGE'].groupby('ticket_created_date_time')['ticket_created_date_time'].value_counts().sort_values()


ticket_created_date_time
2021-06-14      1
2022-04-20      1
2022-12-09      1
2022-04-21      1
2021-04-15      1
             ... 
2022-09-29     73
2022-08-30     74
2020-11-09     79
2022-09-27     81
2022-06-04    134
Name: count, Length: 598, dtype: int64

Merge datasets (for now I have only one zip code in weather, but ideally I can join on date and zip code, which will give me more accurate correlation)

In [8]:
df_weather = pd.read_csv('./virtual_crossing.csv')
df = df.join(df_weather.set_index('datetime'), on='ticket_created_date_time')


Add geohash

In [9]:
import pygeohash as gh
df['geohash6']=df.apply(lambda x: gh.encode(x.X, x.Y, precision=6), axis=1)
df['geohash7']=df.apply(lambda x: gh.encode(x.X, x.Y, precision=7), axis=1)

In [10]:
df.loc[df['issue_Description'] == 'STORM FLOOD/ DRAINAGE'].groupby('ticket_created_date_time')['geohash6'].value_counts().sort_values()

ticket_created_date_time  geohash6
2020-10-01                h9dr3x       1
2022-06-10                h9dr3x       1
2022-06-11                h9dr3z       1
                          h9dpp3       1
2022-06-13                h9drd4       1
                                      ..
2022-09-01                h9dre4      16
2022-06-09                h9dr3z      17
2023-02-02                h9dre4      17
2023-01-20                h9drez      20
2021-11-08                h9drgg      20
Name: count, Length: 2862, dtype: int64

Export dataset for training

In [11]:
# df.groupby()
s_date_geo_count = df.groupby('ticket_created_date_time')['geohash6'].value_counts()
df = df.set_index(['geohash6', 'ticket_created_date_time']).join(s_date_geo_count)


In [12]:
df = df.reset_index()
df = df.loc[df['issue_Description'] == 'STORM FLOOD/ DRAINAGE']
df = df.drop_duplicates(subset=['geohash6', 'ticket_created_date_time'])

df.sample(frac=1, random_state=42)
df_train = df[['geohash6', 'temp', 'precip', 'precipcover', 'count']]
df_training = df_train.head(1700)
df_tail = df_train.tail(1162)
df_validation = df_tail.head(600)
df_test = df_tail.tail(562)
# df_training.to_csv('./training_data.csv')
# df_validation.to_csv('./validation_data.csv')
# df_test.to_csv('./test_data.csv')
df.to_csv('./data.csv')
df.shape

(2862, 64)

In [13]:
# df.loc[df['issue_Description'] == 'STORM FLOOD/ DRAINAGE'].zip_code.value_counts()